In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

c:\Users\prath\OneDrive\Desktop\UCSC\Research\Cloudnet-Research-Project-Repo\ml-models\VMFT_LAD\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
bart_cache_df = pd.read_csv("./Bart_Large_zero_shot_out.csv", header=0, index_col='log_key')
bart_cache = {}
for i, row in bart_cache_df.iterrows():
    bart_cache[i] = row['label']

In [9]:
candidate_labels = ["fault", "unsure", "normal"]
bart = pipeline(model="facebook/bart-large-mnli")

def infer(log: str, log_key: int) -> bool:
        if not log in bart_cache:
            bart_cache[log_key] = bart(log, candidate_labels)["labels"][0] == "fault"
        return bart_cache[log_key]

In [10]:
drain3_state_df = pd.read_csv('../../../../preprocessing/drain3_state.csv', header=0)
drain3_state_df

,log_key,log_template,log_text
0,1,[<:NUM:>.<:NUM:>] br0: port <:*:> entered <:*:...,[.] br0: port entered state
1,2,[<:NUM:>.<:NUM:>] device <:*:> <:*:> promiscuo...,[.] device promiscuous mode
2,3,[<:NUM:>.<:NUM:>] IPv6: ADDRCONF(NETDEV CHANGE...,[.] IPv6: ADDRCONF(NETDEV CHANGE): link becom...
3,4,"[<:NUM:>.<:NUM:>] kvm [<:NUM:>]: vcpu0, guest ...","[.] kvm []: vcpu0, guest rIP: <:HEX:> disabled..."
4,5,[<:NUM:>.<:NUM:>] qemu-system-x86 invoked oom-...,[.] qemu-system-x86 invoked oom-killer: gfp ma...
...,...,...,...
783,784,[ <:NUM:>.<:NUM:>] Code: eb b8 0f 1f <:NUM:> <...,[ .] Code: eb b8 0f 1f f8 0c 4b 8b 0c...
784,785,Starting Daily dpkg database backup service...,Starting Daily dpkg database backup service...
785,786,Finished Daily dpkg database backup service.,Finished Daily dpkg database backup service.
786,787,[ <:NUM:>.<:NUM:>] ? vmx read guest seg ar+<:H...,[ .] ? vmx read guest seg ar+<:HEX:>/<:HEX:> [...


In [11]:
out_df = drain3_state_df.copy()
labels = []

for index, row in tqdm(drain3_state_df.iterrows()):
    labels.append(infer(row["log_text"], row["log_key"]))

out_df["label"] = labels

788it [17:06,  1.30s/it]


In [12]:
out_df

,log_key,log_template,log_text,label
0,1,[<:NUM:>.<:NUM:>] br0: port <:*:> entered <:*:...,[.] br0: port entered state,True
1,2,[<:NUM:>.<:NUM:>] device <:*:> <:*:> promiscuo...,[.] device promiscuous mode,False
2,3,[<:NUM:>.<:NUM:>] IPv6: ADDRCONF(NETDEV CHANGE...,[.] IPv6: ADDRCONF(NETDEV CHANGE): link becom...,False
3,4,"[<:NUM:>.<:NUM:>] kvm [<:NUM:>]: vcpu0, guest ...","[.] kvm []: vcpu0, guest rIP: <:HEX:> disabled...",False
4,5,[<:NUM:>.<:NUM:>] qemu-system-x86 invoked oom-...,[.] qemu-system-x86 invoked oom-killer: gfp ma...,False
...,...,...,...,...
783,784,[ <:NUM:>.<:NUM:>] Code: eb b8 0f 1f <:NUM:> <...,[ .] Code: eb b8 0f 1f f8 0c 4b 8b 0c...,False
784,785,Starting Daily dpkg database backup service...,Starting Daily dpkg database backup service...,False
785,786,Finished Daily dpkg database backup service.,Finished Daily dpkg database backup service.,False
786,787,[ <:NUM:>.<:NUM:>] ? vmx read guest seg ar+<:H...,[ .] ? vmx read guest seg ar+<:HEX:>/<:HEX:> [...,False


In [13]:
out_df.to_csv("./Bart_Large_zero_shot_out.csv", index=False)